<a href="https://colab.research.google.com/github/ousabu/aiousama/blob/master/Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from sklearn import svm,datasets


In [17]:
data=datasets.load_iris()

In [18]:
import pandas as pd


In [19]:
df=pd.DataFrame(data.data,columns=data.feature_names)

In [20]:
df['flowers']=data.target

In [21]:
data.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [22]:
dir(data)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [23]:
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [24]:
df['flowers']=df.flowers.apply(lambda  x:  data.target_names[x])


In [26]:
df[45:60]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flowers
45,4.8,3.0,1.4,0.3,setosa
46,5.1,3.8,1.6,0.2,setosa
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor


In [27]:
from sklearn.model_selection import train_test_split


In [28]:
x=data.data
y=data.target

In [29]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=200)

In [30]:
model=svm.SVC(kernel='rbf',C=10,gamma='auto')

In [31]:
model.fit(x_train,y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [32]:
model.score(x_test,y_test)

1.0

In [33]:
from sklearn.model_selection import cross_val_score

In [36]:
res=cross_val_score(model,x_train,y_train,cv=5)

In [37]:
import numpy as np

np.mean(res)

0.975

In [42]:
kernels=['rbf','linear']
C=[1,10,20]
avg_score={}
for k in kernels:
  for c in C:
    cv_scores=cross_val_score(svm.SVC(kernel=k,C=c,gamma='auto'),x_train,y_train)
    avg_score[k+'_'+str(c)]=np.mean(cv_scores)
avg_score   

{'linear_1': 0.975,
 'linear_10': 0.9666666666666666,
 'linear_20': 0.95,
 'rbf_1': 0.975,
 'rbf_10': 0.975,
 'rbf_20': 0.95}

In [43]:
from sklearn.model_selection import GridSearchCV

In [46]:
clf=GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)

In [47]:
clf.fit(x_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [48]:
clf.score(x_test,y_test)

1.0

In [50]:
clf.best_estimator_

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [51]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [52]:
df_cv=pd.DataFrame(clf.cv_results_)

In [55]:
df_cv

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001125,0.000185,0.000740,0.000157,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.916667,1.000000,1.000000,1.000000,0.958333,0.975000,0.033333,1
1,0.000556,0.000108,0.000380,0.000105,1,linear,"{'C': 1, 'kernel': 'linear'}",0.916667,1.000000,0.958333,1.000000,1.000000,0.975000,0.033333,1
2,0.000629,0.000109,0.000340,0.000033,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.958333,1.000000,0.916667,1.000000,1.000000,0.975000,0.033333,1
3,0.000454,0.000007,0.000310,0.000042,10,linear,"{'C': 10, 'kernel': 'linear'}",0.916667,1.000000,0.916667,1.000000,1.000000,0.966667,0.040825,4
4,0.000543,0.000018,0.000298,0.000009,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.916667,0.958333,0.916667,0.958333,1.000000,0.950000,0.031180,5
5,0.000433,0.000020,0.000285,0.000009,20,linear,"{'C': 20, 'kernel': 'linear'}",0.916667,0.958333,0.916667,0.958333,1.000000,0.950000,0.031180,5


In [57]:
df_cv[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.975000
1,1,linear,0.975000
2,10,rbf,0.975000
3,10,linear,0.966667
4,20,rbf,0.950000
5,20,linear,0.950000


In [58]:
clf.best_score_

0.975

In [ ]:
dir(clf)

In [68]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(data.data, data.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,1,rbf,0.980000


In [70]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [71]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(data.data, data.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 1}
2,logistic_regression,0.966667,{'C': 5}
